### Import

In [10]:
import os
import time
import sys
from pathlib import Path
import numpy as np
from tqdm import tqdm
from timm import create_model
import torch
import torch.nn as nn
import torch.utils.data as data
from torch.autograd import Variable
import torchvision
from torchvision.transforms import ToTensor, Compose, Resize, Normalize
import torchvision.models as models
import matplotlib.pyplot as plt

### Data



In [11]:
transform = Compose([
    Resize((224, 224)),  # 調整到模型需求的輸入大小
    ToTensor(),
    Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])  # 常見正規化數值
])

train_ds = torchvision.datasets.ImageFolder('../0_data/0_rawDataSet/RSCD dataset-1million/train', transform=transform)
valid_ds = torchvision.datasets.ImageFolder('../0_data/0_rawDataSet/RSCD dataset-1million/vali_20k', transform=transform)
test_ds  = torchvision.datasets.ImageFolder('../0_data/0_rawDataSet/RSCD dataset-1million/test_50k', transform=transform)

### Model import init

In [12]:
sys.path.append(str(Path("0_utils") ))

# 匯入模型初始化模組
from models_initializer import initialize_all_models

# 初始化模型
NUM_CLASSES = len(train_ds.class_to_idx)
models = initialize_all_models(NUM_CLASSES, pretrained=False)


c:\Users\YunChung\miniconda3\envs\eenv\lib\site-packages\torchvision\models\_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
c:\Users\YunChung\miniconda3\envs\eenv\lib\site-packages\torchvision\models\_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=None`.
  warnings.warn(msg)
c:\Users\YunChung\miniconda3\envs\eenv\lib\site-packages\torchvision\models\googlenet.py:47: FutureWarning: The default weight initialization of GoogleNet will be changed in future releases of torchvision. If you wish to keep the old behavior (which leads to long initialization times due to scipy/scipy#11299), please set init_weights=True.
  warnings.warn(
c:\Users\YunChung\miniconda3\envs\eenv\lib\site-packages\timm\models\_factory.py:117: UserWarning:

### Training Parameters

| Model           | 參數                         |
|:----------------|:-----------------------------|
| alexnet         | None                         |
| vgg16           | dropout_rate, use_batch_norm |
| googlenet       | aux_logits, transform_input  |
| resnet18        | None                         |
| squeezenet      | None                         |
| mobilenet_v2    | width_mult, round_nearest    |
| efficientnet_b0 | None                         |
| swin_t          | drop_path_rate               |
| xception        | dropout_rate                 |
| convnext_tiny   | drop_path_rate               |
| mnasnet         | None                         |

In [13]:
# Model
model = alexnet

EPOCHS = 1
BATCH_SIZE = 256
LEARNING_RATE = 1e-3   
optimizer = torch.optim.Adam(model.parameters(), lr=LEARNING_RATE) 
loss_func = nn.CrossEntropyLoss()

num_workers = 4

### Setup

In [14]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu') 
if torch.cuda.is_available():
    model.cuda()

### Train

In [ ]:
# 儲存訓練損失與準確度
train_losses = []
train_accuracies = []

# 訓練
start_time = time.time()
for epoch in range(EPOCHS):
    print(f"Epoch {epoch + 1}/{EPOCHS}")
    
    model.train()
    train_loss = 0.0
    correct_train = 0
    total_train = 0

    # 使用 tqdm 包裹 train_loader
    train_bar = tqdm(enumerate(train_loader), total=len(train_loader), desc="Training")
    for step, (x, y) in train_bar:
        x, y = x.to(device), y.to(device)

        # Forward
        output = model(x)
        loss = loss_func(output, y)

        # Backpropagation
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        # 累計損失與準確度
        train_loss += loss.item() * x.size(0)
        _, preds = torch.max(output, 1)
        correct_train += (preds == y).sum().item()
        total_train += y.size(0)

        # 更新進度條
        train_bar.set_postfix({"loss": loss.item()})

    # 計算單個 epoch 的平均損失與準確度
    epoch_loss = train_loss / len(train_ds)
    epoch_accuracy = correct_train / total_train

    train_losses.append(epoch_loss)
    train_accuracies.append(epoch_accuracy)

    print(f"Epoch {epoch + 1}: Train Loss: {epoch_loss:.4f}, Train Accuracy: {epoch_accuracy:.4f}")

training_time = time.time() - start_time

# 繪製損失與準確度圖表
epochs = range(1, EPOCHS + 1)

plt.figure(figsize=(12, 5))

# 損失圖
plt.subplot(1, 2, 1)
plt.plot(epochs, train_losses, label="Train Loss")
plt.xlabel("Epochs")
plt.ylabel("Loss")
plt.title("Train Loss")
plt.legend()

# 準確度圖
plt.subplot(1, 2, 2)
plt.plot(epochs, train_accuracies, label="Train Accuracy")
plt.xlabel("Epochs")
plt.ylabel("Accuracy")
plt.title("Train Accuracy")
plt.legend()

plt.tight_layout()
plt.show()


Epoch 1/1


Training:   6%|▌         | 213/3746 [04:56<1:22:31,  1.40s/it, loss=1.21] 

### Save Model


In [9]:
# 測試準確度
test_x, test_y = next(iter(test_loader))
test_x, test_y = test_x.to(device), test_y.to(device)
test_output = model(test_x).argmax(1)
accuracy = (test_output == test_y).sum().item() / BATCH_SIZE

# 模型名稱
model_name = type(model).__name__

# 檔案名稱包含訓練時間
file_name = f"{model_name}_ACC{accuracy:.2f}_E{EPOCHS}_BS{BATCH_SIZE}_LR{LEARNING_RATE:.0e}_T{int(training_time // 3600):02d}h{int((training_time % 3600) // 60):02d}m{int(training_time % 60):02d}s.pt"

# 儲存路徑
save_path = os.path.join('../2_results/0_weight', file_name)
torch.save(model, save_path)

print(f"Model saved as: {file_name}")

Model saved as: AlexNet_ACC0.54_E1_BS256_LR1e-03_T00h30m46s.pt
